In [1]:
import numpy as np
import pandas as pd
import datetime as dt

In [2]:
df = pd.read_csv('C:/Users/ggibs/data/US_Accidents_Dec19.csv', parse_dates = ['Start_Time', 'End_Time'])  

In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2974335 entries, 0 to 2974334
Data columns (total 49 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   ID                     object        
 1   Source                 object        
 2   TMC                    float64       
 3   Severity               int64         
 4   Start_Time             datetime64[ns]
 5   End_Time               datetime64[ns]
 6   Start_Lat              float64       
 7   Start_Lng              float64       
 8   End_Lat                float64       
 9   End_Lng                float64       
 10  Distance(mi)           float64       
 11  Description            object        
 12  Number                 float64       
 13  Street                 object        
 14  Side                   object        
 15  City                   object        
 16  County                 object        
 17  State                  object        
 18  Zipcode               

In [5]:
# % null
df.isna().sum()/2974335*100

ID                        0.000000
Source                    0.000000
TMC                      24.478446
Severity                  0.000000
Start_Time                0.000000
End_Time                  0.000000
Start_Lat                 0.000000
Start_Lng                 0.000000
End_Lat                  75.521554
End_Lng                  75.521554
Distance(mi)              0.000000
Description               0.000034
Number                   64.471722
Street                    0.000000
Side                      0.000000
City                      0.002791
County                    0.000000
State                     0.000000
Zipcode                   0.029586
Country                   0.000000
Timezone                  0.106343
Airport_Code              0.191337
Weather_Timestamp         1.234057
Temperature(F)            1.884892
Wind_Chill(F)            62.286965
Humidity(%)               1.989453
Pressure(in)              1.618580
Visibility(mi)            2.208595
Wind_Direction      

In [72]:
# End_Lat & _Lng are 75.5% null
# Can records have Distance without End coordinates?  Yes
df[(df['End_Lat'].isnull() == True) & (df['Distance(mi)'] > 0.1)].shape

(132433, 49)

In [75]:
# Can Distance be zero and End_Lat not null?  Yes
df[(df['End_Lat'].notnull() == True) & (df['Distance(mi)'] == 0)].shape
# End_Lat and End_Lng are not required.

(152280, 49)

In [ ]:
# Street Number is 64.5% null and not required, considering highways and ramps, etc., do not have a street address.

In [77]:
# TMC is 24.5% null, while 63.5% of available points are in one category, and likely not beneficial.
# % in TMC categories
df.TMC.value_counts()/2974335*100

201.0    63.544860
241.0     7.365377
245.0     1.207631
229.0     0.761616
203.0     0.564563
222.0     0.422010
244.0     0.375311
406.0     0.318290
246.0     0.213493
202.0     0.203306
343.0     0.195909
247.0     0.155194
236.0     0.071142
206.0     0.038899
248.0     0.034024
339.0     0.026023
341.0     0.016945
336.0     0.002723
200.0     0.002219
239.0     0.001816
351.0     0.000202
Name: TMC, dtype: float64

In [84]:
# Exclude End_Lat, End_Lng, Number (street), and TMC columns
df2 = df.drop(['End_Lat', 'End_Lng', 'Number', 'TMC'], axis = 1)
df2.shape

(2974335, 45)

In [85]:
# Drop remaining NaN records
df_nonan = df2.dropna()
df_nonan.shape

(779721, 45)

In [115]:
# Current subset is 26% of original file
round(779721 / 2974335 *100,1)

26.2

In [92]:
# Create a traffic duration of minutes
#df_nonan['Duration'] = df_nonan['End_Time'] - df_nonan['Start_Time']
df_nonan['Duration'] = df_nonan.apply(lambda x: x['End_Time'] - x['Start_Time'], axis = 1)
# Convert to minutes
df_nonan['Duration'] = df_nonan['Duration']/np.timedelta64(1,'m')

C:\Users\ggibs\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\ggibs\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [100]:
# A few records are exactly 30 min to the second
df_nonan['Duration'][df_nonan['Duration'] == 30.0000].count()

2122

In [106]:
# Does it correlate to Severity?  No
df_nonan['Duration'].corr(df_nonan['Severity'])

0.010502699827644403

In [105]:
# Does Distance correlate to Severity?  No
df_nonan['Distance(mi)'].corr(df_nonan['Severity'])

0.1777027071605262

In [111]:
# Can rain during good weather indicator
subset = (df_nonan['Weather_Condition'].isin(['Fair', 'Clear'])) & (df_nonan['Precipitation(in)']>0.1) 
WCFair = df_nonan[['Weather_Condition', 'Precipitation(in)']][subset]
print(WCFair.head(10))

       Weather_Condition  Precipitation(in)
145692              Fair               0.12
204631             Clear               0.17
204657             Clear               9.85
231212             Clear               0.20
243012             Clear               0.93
243016             Clear               0.93
302453              Fair               0.39
327351             Clear               9.95
370781             Clear               9.99
373982             Clear               9.97


In [ ]:
# Original file was over 60% null in Precipitation(in) and Wind_Chill(F)
# Here is an alternative effort to salvage a greater proportion of records
# Plan B:  Calculate mean and standard deviation of weather categories during that month and zipcode and replace NaNs

In [3]:
df['Month'] = df['Start_Time'].dt.month
# df['Day'] = df['Start_Time'].dt.day
# df['Hour'] = df['Start_Time'].dt.hour

In [4]:
# Zipcode is null in only 0.03% of records, remove null zipcode rows
df = df[df['Zipcode'].notna()]

In [5]:
# Shorten zipcode for larger areas
df['Zip2'] = df['Zipcode'].str[:2]

In [6]:
# Group data by Zipcode2 and Month for regional weather similarities
df_group = df.groupby(['Zip2', 'Month'], as_index = False)

In [7]:
# Calculate mean and std deviation of weather measurements by zip-month grouping
monthly_precip = df_group['Precipitation(in)'].agg(['mean', 'std']).reset_index()
monthly_chill = df_group['Wind_Chill(F)'].agg(['mean', 'std']).reset_index()
monthly_speed = df_group['Wind_Speed(mph)'].agg(['mean', 'std']).reset_index()

In [8]:
# Create an index by zip-month combo
monthly_precip['Zip2Month'] = monthly_precip['Zip2'] + monthly_precip['Month'].astype(str)
mp = monthly_precip[['Zip2Month', 'mean', 'std']].set_index('Zip2Month')

monthly_chill['Zip2Month'] = monthly_chill['Zip2'] + monthly_chill['Month'].astype(str)
mc = monthly_chill[['Zip2Month', 'mean', 'std']].set_index('Zip2Month')

monthly_speed['Zip2Month'] = monthly_speed['Zip2'] + monthly_speed['Month'].astype(str)
ms = monthly_speed[['Zip2Month', 'mean', 'std']].set_index('Zip2Month')

In [ ]:
# Fill weather NaNs with above means and standard deviations
for i in df.index:
    row = df.loc[i,'Zip2'] + df.loc[i,'Month'].astype(str)
    try:
        if pd.isnull(df.loc[i, 'Precipitation(in)']):
            df.loc[i, 'Precipitation(in)'] = np.random.normal(mp.loc[row, 'mean'], mp.loc[row, 'std'], 1)
    except:
        continue
    try:
        if pd.isnull(df.loc[i, 'Wind_Chill(F)']):
            df.loc[i, 'Wind_Chill(F)'] = np.random.normal(mc.loc[row, 'mean'], mc.loc[row, 'std'], 1)
    except:
        continue
    try:
        if pd.isnull(df.loc[i, 'Wind_Speed(mph)']):
            df.loc[i, 'Wind_Speed(mph)'] = np.random.normal(ms.loc[row, 'mean'], ms.loc[row, 'std'], 1)
    except:
        continue
    else:
        continue

In [ ]:
print(df[['Precipitation(in)', 'Wind_Chill(F)', 'Wind_Speed(mph)']].head(50))

In [138]:
# remove outliers
from scipy import stats

In [179]:
z_scores = stats.zscore(df2)
abs_z_scores = np.abs(z_scores)
filtered_entries = (abs_z_scores < 3).all(axis=1)
new_df = df2[filtered_entries]

In [180]:
new_df.shape

(2663278, 10)

In [9]:
3000000*.25

750000.0

In [ ]:
'''
API alternative - This errors out due to too many record requests
RapidAPI for Dark Sky would allow unlimited records for $0.001.  67% of 3M records is 2M records and would cost $2K

import requests

url = 'http://api.worldweatheronline.com/premium/v1/past-weather.ashx'
API_KEY = '*****'
loc = str(df['Start_Lat'][0]) + ', ' + str(df['Start_Lng'][0])
d = str(df.Start_Time[0].date())
params = dict(key = API_KEY, q = loc, format = 'json', date = d, includelocation = 'yes', tp=1 )

res = requests.get(url, params)

json = res.json()

for i in df:
    if str(df.loc[i, 'Wind_Chill(F)']) == 'nan' or str(df.loc[i, 'Wind_Speed(mph)']) == 'nan' or str(df.loc[i, 'Precipitation(in)']) == 'nan':
            loc = str(df['Start_Lat'][i]) + ', ' + str(df['Start_Lng'][i])
            d = str(df.Start_Time[i].date())
            params = dict(key = API_KEY, q = loc, format = 'json', date = d, includelocation = 'yes', tp=1 )
            res = requests.get(url, params)
            json = res.json()
            if df.Start_Time[i].time().hour == 23:
                h = 23
            elif df.Start_Time[i].time().minute > 30:
                h = df.Start_Time[i].time().hour + 1
            else:
                h = df.Start_Time[i].time().hour

            if str(df.loc[i, 'Wind_Chill(F)']) == 'nan':
                df.loc[i,'Wind_Chill(F)'] = int(json['data']['weather'][0]['hourly'][h]['WindChillF'])
            if str(df.loc[i, 'Wind_Speed(mph)']) == 'nan':
                df.loc[i,'Wind_Speed(mph)'] = int(json['data']['weather'][0]['hourly'][h]['windspeedMiles'])
            if str(df.loc[i, 'Precipitation(in)']) == 'nan':
                df.loc[i,'Precipitation(in)'] = float(json['data']['weather'][0]['hourly'][h]['precipInches'])
    else:
        continue
'''